In [1]:
import joblib
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt



In [4]:
# Definir rutas
ruta_actual = os.getcwd()
carpeta_mediapipe = os.path.dirname(ruta_actual)
ruta_raiz = os.path.dirname(carpeta_mediapipe)

ruta_npy = os.path.join(ruta_actual, 'datos', 'classes.npy')
ruta_scaler = os.path.join(ruta_actual, 'datos', 'scaler_X.pkl')
ruta_modelo = os.path.join(ruta_actual, 'model', 'exercise_model.h5')
ruta_csv = os.path.join(carpeta_mediapipe, 'Datos_Videos_Pose', 'pose_data.csv')

# Cargar el dataset
data = pd.read_csv(ruta_csv)

# Preparar entradas (Keypoints)
num_keypoints = 33 * 3
X = data.iloc[:, :num_keypoints].values

# Las siguientes 11 columnas incluyen:
# 10 características continuas (ángulos, distancias) + 1 tipo de ejercicio (última columna)
y_continuas = data.iloc[:, num_keypoints:(data.shape[1] - 1)].values
y_clase = data.iloc[:, -1].values

# Codificar la etiqueta del tipo de ejercicio
encoder = LabelEncoder()
y_clase_encoded = encoder.fit_transform(y_clase)
num_clases = len(encoder.classes_)

# Normalizar las entradas
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# Reshape para LSTM
X_scaled = X_scaled[:, :, np.newaxis]  # Añadir una dimensión para el LSTM

# Definir función para crear el modelo
def create_model(input_shape, num_clases, learning_rate=0.001):
    input_layer = Input(shape=input_shape)

    # Capas LSTM con Dropout
    x = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(input_layer)

    # Salidas continuas (10 características)
    salidas_continuas = Dense(10, activation='linear', name='salidas_continuas')(x)

    # Salida de tipo de ejercicio (multiclase)
    salida_clase = Dense(num_clases, activation='softmax', name='salida_clase')(x)

    model = Model(inputs=input_layer, outputs=[salidas_continuas, salida_clase])

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss={
                      'salidas_continuas': 'mean_squared_error',
                      'salida_clase': 'sparse_categorical_crossentropy'
                  },
                  metrics={
                      'salidas_continuas': 'mae',
                      'salida_clase': 'accuracy'
                  })
    return model

# Inicializar KFold y métricas
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

all_loss = []
all_mae = []
all_mse = []
all_rmse = []
all_r2 = []
all_accuracy = []
all_precision = []
all_recall = []
all_f1 = []

# Matrices para almacenar la historia del entrenamiento
n_epochs = 50  # Número de épocas
all_train_loss = np.full((n_splits, n_epochs), np.nan)  
all_val_loss = np.full((n_splits, n_epochs), np.nan)
all_train_accuracy = np.full((n_splits, n_epochs), np.nan)
all_val_accuracy = np.full((n_splits, n_epochs), np.nan)

# Validación cruzada
for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
    print(f"Entrenando fold {fold + 1}...")

    X_train_fold, X_val_fold = X_scaled[train_index], X_scaled[val_index]
    y_continuas_train_fold, y_continuas_val_fold = y_continuas[train_index], y_continuas[val_index]
    y_clase_train_fold, y_clase_val_fold = y_clase_encoded[train_index], y_clase_encoded[val_index]

    model = create_model(input_shape=(X_train_fold.shape[1], X_train_fold.shape[2]), num_clases=num_clases, learning_rate=0.0005)

    tensorboard_callback = TensorBoard(log_dir=os.path.join(ruta_actual, 'logs', f'fold_{fold + 1}'), histogram_freq=1)
    early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

    history = model.fit(X_train_fold, {'salidas_continuas': y_continuas_train_fold, 'salida_clase': y_clase_train_fold},
                        validation_data=(X_val_fold, {'salidas_continuas': y_continuas_val_fold, 'salida_clase': y_clase_val_fold}),
                        epochs=n_epochs, batch_size=32, callbacks=[tensorboard_callback, early_stopping_callback, reduce_lr_callback])

    # Almacenar la historia del entrenamiento como matrices NumPy
    epochs_completed = len(history.history['loss'])
    all_train_loss[fold, :epochs_completed] = history.history['loss']
    all_val_loss[fold, :epochs_completed] = history.history['val_loss']
    all_train_accuracy[fold, :epochs_completed] = history.history['salida_clase_accuracy']
    all_val_accuracy[fold, :epochs_completed] = history.history['val_salida_clase_accuracy']

    # Evaluar el modelo en el conjunto de validación
    scores = model.evaluate(X_val_fold, {'salidas_continuas': y_continuas_val_fold, 'salida_clase': y_clase_val_fold})
    print(f"Fold {fold + 1}: Loss={scores[0]}, MAE={scores[2]}, Accuracy={scores[1]}")

    # Calcular métricas adicionales para las salidas continuas
    y_pred_continuas = model.predict(X_val_fold)[0]
    mae = mean_absolute_error(y_continuas_val_fold, y_pred_continuas)
    mse = mean_squared_error(y_continuas_val_fold, y_pred_continuas)
    rmse = np.sqrt(mse)
    r2 = 1 - mse / np.var(y_continuas_val_fold)

    all_loss.append(scores[0])
    all_mae.append(mae)
    all_mse.append(mse)
    all_rmse.append(rmse)
    all_r2.append(r2)
    all_accuracy.append(scores[1])

    # Calcular métricas de clasificación
    y_pred_clase = np.argmax(model.predict(X_val_fold)[1], axis=1)
    precision = precision_score(y_clase_val_fold, y_pred_clase, average='weighted')
    recall = recall_score(y_clase_val_fold, y_pred_clase, average='weighted')
    f1 = f1_score(y_clase_val_fold, y_pred_clase, average='weighted')

    all_precision.append(precision)
    all_recall.append(recall)
    all_f1.append(f1)

    # Imprimir matriz de confusión
    cm = confusion_matrix(y_clase_val_fold, y_pred_clase, labels=np.unique(y_clase_val_fold))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=encoder.classes_)
    plt.figure(figsize=(12, 8))
    disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
    plt.title(f'Matriz de Confusión ')
    plt.tight_layout()
    plt.show()



Entrenando fold 1...
Epoch 1/50
499/900 ━━━━━━━━━━━━━━━━━━━━ 18s 47ms/step - loss: 7210.8940 - salida_clase_accuracy: 0.2088 - salidas_continuas_mae: 60.4643

In [ ]:
# Promediar las historias del entrenamiento
mean_train_loss = np.mean(all_train_loss, axis=0)
mean_val_loss = np.mean(all_val_loss, axis=0)
mean_train_accuracy = np.mean(all_train_accuracy, axis=0)
mean_val_accuracy = np.mean(all_val_accuracy, axis=0)

# Graficar los promedios
plt.figure(figsize=(12, 6))

# Promedio de Loss
plt.subplot(1, 2, 1)
plt.plot(mean_train_loss, label='Training Loss Promedio')
plt.plot(mean_val_loss, label='Validation Loss Promedio')
plt.title('Promedio de Loss durante el Entrenamiento y la Validación')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Promedio de Accuracy
plt.subplot(1, 2, 2)
plt.plot(mean_train_accuracy, label='Training Accuracy Promedio')
plt.plot(mean_val_accuracy, label='Validation Accuracy Promedio')
plt.title('Promedio de Accuracy durante el Entrenamiento y la Validación')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Imprimir métricas promedio de todos los folds
print("\nMétricas promedio de todos los folds:")
print(f"Loss: {np.mean(all_loss)}")
print(f"MAE: {np.mean(all_mae)}")
print(f"MSE: {np.mean(all_mse)}")
print(f"RMSE: {np.mean(all_rmse)}")
print(f"R^2: {np.mean(all_r2)}")
print(f"Accuracy: {np.mean(all_accuracy)}")
print(f"Precisión: {np.mean(all_precision)}")
print(f"Recall: {np.mean(all_recall)}")
print(f"F1-Score: {np.mean(all_f1)}")

# Guardar el modelo
model.save(ruta_modelo)
joblib.dump(scaler_X, ruta_scaler)
np.save(ruta_npy, encoder.classes_)
